In [3]:
import pandas as pd
import re
from keybert import KeyBERT
import yfinance as yf
import time
import random
from collections import Counter
from nltk.stem import WordNetLemmatizer
import numpy as np

news_raw = pd.read_csv("C:/projectnasdaq/news_raw.csv", encoding='latin1') # 뉴스 raw data
nasdaq_stock = pd.read_csv("C:/projectnasdaq/nasdaq_stocks.csv", encoding='latin1') # stock_list data
rep_stock = pd.read_csv("C:/projectnasdaq/nasdaq_stocks_refine_total.csv", encoding='latin1') # 대표단어 csv

# 테스트 용도로 데이터 1000개
news_raw = news_raw.head(1000)
news_raw = news_raw[['news_id','title','summary']]
nasdaq_stock = nasdaq_stock[['pk','symbol','name']]
rep_stock = rep_stock[['pk','symbol','name','name_a']]

# new_raw 데이터 전처리
news_raw['summary'] = news_raw['summary'].str.lower() #소문자화
news_raw['summary'] = news_raw['summary'].apply(lambda x: re.sub('[^a-zA-Z\d&]', ' ', str(x)).strip())
self_stop_words = {'bloomberg'}
news_raw['summary'].replace(self_stop_words, '', regex=True, inplace=True)

# news_raw 데이터 토큰화
news_raw['tokenize'] = 0
news_raw['tokenize'] = news_raw['summary'].str.split(" ")
news_raw['tokenize'] = news_raw['tokenize'].apply(lambda x: [i for i in x if i != "" and i != " "])

news_raw[] = 0
news_raw['keyword'] = 0 # keybert로 뽑아낸 키워드 컬럼
news_raw['keyword2'] = 0

news_raw

,news_id,title,summary,tokenize,company_name,keyword,keyword2
0,345585,Nestle Outshines Rivals With Revenue Growth Le...,nestle sa sailed past ailing consumer goo...,"[nestle, sa, sailed, past, ailing, consumer, g...",0,0,0
1,345586,Total Makes Surprise Profit as Trading Gains O...,total se made a surprise profit after ve...,"[total, se, made, a, surprise, profit, after, ...",0,0,0
2,345590,Airbus Follows Boeing in Paring Output to Weat...,airbus se cut back wide body jet producti...,"[airbus, se, cut, back, wide, body, jet, produ...",0,0,0
3,345591,Virus Uptick Imperils South Europe's Nascent T...,nadine scheiner s efforts to travel from ...,"[nadine, scheiner, s, efforts, to, travel, fro...",0,0,0
4,345595,PG&E Fire Insurance Costs Skyrocket After Bank...,pg&e corp is finding it very costly to b...,"[pg&e, corp, is, finding, it, very, costly, to...",0,0,0
...,...,...,...,...,...,...,...
995,443773,Localiza to Form $9 Billion Car-Rental Giant W...,localiza rent a car sa and unidas two of...,"[localiza, rent, a, car, sa, and, unidas, two,...",0,0,0
996,443778,Mendoza Province Is First to Restructure After...,argentina s mendoza province has reached ...,"[argentina, s, mendoza, province, has, reached...",0,0,0
997,443779,Manufacturing Keeps German Economy on a Recove...,german factories kept europe s biggest ec...,"[german, factories, kept, europe, s, biggest, ...",0,0,0
998,443780,ECB Must Limit Emergency Powers to Temporary C...,the european central bank risks legal tro...,"[the, european, central, bank, risks, legal, t...",0,0,0


In [ ]:
# list에 담아주기
tokenize_list = news_raw['tokenize']
rep_list = rep_stock['name_a'].str.split(",")

## 회사 이름 추출 => 추출 완료 ( company_word.csv 에 저장 )

In [ ]:
# 회사이름 추출 데이터 저장용도
company_word = pd.DataFrame(columns=['news_id', 'company_word','pk'])

# 대표단어 csv파일에서 대표단어가 포함되면 맵핑
i = 0

for token_num, token in enumerate(tokenize_list):  # news_raw data 토큰화
    for rep_num, rep in enumerate(rep_list):  # 대표단어 csv파일에서 대표단어에 해당
        i = i + 1
        if len(rep) == 1:
            if rep[0] in token:
                company_word.loc[i] = [news_raw['news_id'][token_num], rep[0], rep_stock['pk'][rep_num]]

        elif len(rep) == 2:  # 대표단어가 2개로 된 단어일 때
            if rep[0] in token:  # 대표단어의 첫번째 단어가 org단어에 있으면
                found = token.index(rep[0])  # 대표단어의 첫번째 단어와 일치하는 org단어의 인덱스 위치 번호
                try:
                    search = found + 1  # stocklist의 첫번째 단어가 org에 포함됐을때 그 다음 단어
                    search_found = token[search]  # org의 (+1을 한) 다음 단어에 해당
                    if rep[1] == search_found:
                        company_word.loc[i] = [news_raw['news_id'][token_num], rep[0] + " " + rep[1],
                                                 rep_stock['pk'][rep_num]]
                except IndexError:
                    continue

        elif len(rep) == 3:
            if rep[0] in token:
                try:
                    search_found = token[token.index(rep[0]) + 1]
                    if rep[1] == search_found:
                        two_found = token[token.index(rep[0]) + 2]
                        if rep[2] == two_found:
                            company_word.loc[i] = [news_raw['news_id'][token_num],
                                                     rep[0] + " " + rep[1] + " " + rep[2], rep_stock['pk'][rep_num]]
                except IndexError:
                    continue

company_word = pd.merge(company_word, nasdaq_stock, how='left', left_on='pk', right_on='pk')
#company_word = pd.read_csv('C:/projectnasdaq/project2/company_word.csv')